<a href="https://colab.research.google.com/github/xueyil9/baseflow_pypi/blob/master/2_GSLB_PCHIP_DeltaWTE_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
measurements = pd.read_csv('GSLB_1900-2023_TS_with_aquifers.csv')

In [6]:
from scipy.interpolate import PchipInterpolator
import numpy as np

# Convert 'Date' column to datetime objects if not already done
measurements['Date'] = pd.to_datetime(measurements['Date'])

# Group data by 'Well'
grouped = measurements.groupby('Well_ID')

# Iterate through each well's data
interpolated_data = []
for well, data in grouped:
    if len(data) >= 2:
        # Create daily date range for the well's data period
        date_range = pd.date_range(start=data['Date'].min(), end=data['Date'].max(), freq='D')

        # Prepare x (date) and y (measurement) data for interpolation
        x = data['Date'].astype(np.int64) // 10**9  # Convert datetime to Unix timestamps in seconds
        y = data['WTE']

        # Create the PCHIP interpolator
        f = PchipInterpolator(x, y)

        # Interpolate for the full daily range
        x_new = date_range.astype(np.int64) // 10**9
        y_new = f(x_new)

        # Create a new DataFrame with daily interpolated data
        interpolated_df = pd.DataFrame({'Date': date_range, 'Well_ID': well, 'WTE': y_new})
        interpolated_data.append(interpolated_df)
    else:
        # Append original data if there are less than 2 data points
        interpolated_data.append(data)

# Concatenate all the interpolated dataframes
final_measurements = pd.concat(interpolated_data)

print(final_measurements.head())

   AquiferID          Well_ID       Date      WTE State
0        1.0  381033113480701 2012-09-06  7092.99    UT
1        1.0  381037113474001 2012-09-06  7175.95    UT
2        1.0  381152113442801 1995-11-22  6200.00    UT
3        1.0  381236113485601 2014-07-23  7151.00    UT
0        NaN  382113113435401 2008-09-03  5395.95   NaN


In [7]:
if 'AquiferID' in final_measurements.columns:
    final_measurements = final_measurements.drop('AquiferID', axis=1)
if 'State' in final_measurements.columns:
    final_measurements = final_measurements.drop('State', axis=1)
final_measurements.head()

,Well_ID,Date,WTE
0,381033113480701,2012-09-06,7092.99
1,381037113474001,2012-09-06,7175.95
2,381152113442801,1995-11-22,6200.00
3,381236113485601,2014-07-23,7151.00
0,382113113435401,2008-09-03,5395.95


In [18]:
# Group the final_measurements DataFrame by 'Well_ID'
grouped_wells = final_measurements.groupby('Well_ID')

# Store the results
delta_wte_data = []

# Iterate through each well
for well, data in grouped_wells:
    # Get the first WTE value for the well
    first_wte = data['WTE'].iloc[0]

    # Calculate Delta_WTE for each row in the well's data
    data['Delta_WTE'] = data['WTE'] - first_wte
    delta_wte_data.append(data)

# Concatenate the results back into a single DataFrame
final_measurements_delta = pd.concat(delta_wte_data)
final_measurements_delta.sort_values(by='Well_ID', inplace=True)

# Reduce precision of float columns
float_columns = final_measurements_delta.select_dtypes(include=['float64', 'float32','float16']).columns
for col in float_columns:
    final_measurements_delta[col] = final_measurements_delta[col].round(3).astype('float32')

print(final_measurements_delta.head())

              Well_ID       Date          WTE  Delta_WTE
0     381033113480701 2012-09-06  7092.990234      0.000
1     381037113474001 2012-09-06  7175.950195      0.000
2     381152113442801 1995-11-22  6200.000000      0.000
3     381236113485601 2014-07-23  7151.000000      0.000
3551  382113113435401 2018-05-25  5397.996094      2.046


In [20]:
# Export the final_measurements_delta DataFrame to a CSV file
final_measurements_delta.to_csv('final_measurements_delta.csv', index=False)